In [ ]:
from sdk.ample import Ample

from sdk.graphs.random_graph import RandomGraph
#TODO change to just models in file sturcture
from sdk.models.models import GCN_Model, GAT_Model, GraphSAGE_Model, GIN_Model, GCN_MLP_Model, MLP_Model, Edge_Embedding_Model, Interaction_Net_Model
from sdk.graph_tracer import GraphTracer

%load_ext autoreload
%autoreload 2


In [ ]:
#Import neural lam
ample = Ample()
edge = True
#TODO fix this : If model does not use edges, dont set edges to be true - will brrak things 
graph = RandomGraph(num_nodes=10, avg_degree=1, num_channels=32, graph_precision="FLOAT_32",edge_dim=32,edges = edge) #TODO add var

In [ ]:
#Trace model, find the constituent models e.g GCN, Interaction net and create a data flow graph

import sys
sys.path.insert(0,'/home/aw1223/ip/worktree_a/imports/neural-lam')

import torch

from neural_lam.models.graph_lam import GraphLAM
from neural_lam.weather_dataset import WeatherDataset

import argparse
# Manually create the args Namespace object with the desired parameters
args = argparse.Namespace(
    dataset='meps_example',
    model='graph_lam',
    subset_ds=0,
    seed=42,
    n_workers=4,
    epochs=200,
    batch_size=4,
    load=None,
    restore_opt=0,
    precision=32,
    graph='1level',
    hidden_dim=64,
    hidden_layers=1,
    processor_layers=4,
    mesh_aggr='sum',
    output_std=0,
    ar_steps=1,
    control_only=0,
    loss='wmse',
    step_length=3,
    lr=0.001,
    val_interval=1,
    eval='test',
    n_example_pred=1
)
print('Neural LAM Args', args)
#Change to neural-lam directory
%cd /home/aw1223/ip/worktree_a/imports/neural-lam
# Initialize your model
model = GraphLAM(args)


#Remove this as its part of training not testing - just need one batch
#Then send to device
eval_loader = torch.utils.data.DataLoader(
                WeatherDataset(
                    args.dataset,
                    # pred_length=max_pred_length,
                    split="test",
                    subsample_step=args.step_length,
                    subset=bool(args.subset_ds),
                ),
                args.batch_size,
                shuffle=False,
                num_workers=args.n_workers,
            )

for batch in eval_loader:
    graph = batch  # Trigger forward pass
    break 
# %cd -
ample = Ample()
#Use fx graph so that no need for eval loader and to_device is used properly
model.to_device('ample',graph)

#Send data before to load device
#graph.todevice('ample')



In [ ]:
%cd /home/aw1223/ip/worktree_a/imports/neural-lam
ample.compile(model, eval_loader=eval_loader, trace_mode = 'hooks')


In [ ]:
# ample.to_device()

In [ ]:
# import pytorch_lightning as pl
# trainer = pl.Trainer(
#       max_epochs=args.epochs,
#       deterministic=True,
#       strategy="ddp_notebook",
#       # accelerator=device_name,
#       # logger=logger,
#       log_every_n_steps=1,
#       # callbacks=[checkpoint_callback],
#       check_val_every_n_epoch=args.val_interval,
#       precision=args.precision,
#   )



# trainer.test(model=model, dataloaders=eval_loader)


In [ ]:
#Use initalize to construct memory map for each model and then string together the dataflow between models

In [ ]:
#Program Agile using device